In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

path = '/content/Raw_Data/'

cpi = pd.read_csv(path + 'CPIAUCSL.csv', parse_dates=['observation_date'], index_col='observation_date')
fed = pd.read_csv(path + 'FEDFUNDS.csv', parse_dates=['observation_date'], index_col='observation_date')
wages = pd.read_csv(path + 'CES0500000003.csv', parse_dates=['observation_date'], index_col='observation_date')
unemployment = pd.read_csv(path + 'UNRATE.csv', parse_dates=['observation_date'], index_col='observation_date')
savings = pd.read_csv(path + 'PSAVERT.csv', parse_dates=['observation_date'], index_col='observation_date')
pce = pd.read_csv(path + 'PCE.csv', parse_dates=['observation_date'], index_col='observation_date')
investment = pd.read_csv(path + 'GPDI.csv', parse_dates=['observation_date'], index_col='observation_date')
gdp = pd.read_csv(path + 'GDP.csv', parse_dates=['observation_date'], index_col='observation_date')

cpi.columns = ['CPI']
fed.columns = ['Federal Funds Rate']
wages.columns = ['Average Hourly Earnings']
unemployment.columns = ['Unemployment']
savings.columns = ['Personal Savings Rate']
pce.columns = ['PCE']
investment.columns = ['GPDI']
gdp.columns = ['GDP']

df = cpi.join([fed, wages, unemployment, savings, pce, investment, gdp])
df.dropna(inplace=True)

df = df[df.index >= '2006-04-01']
df['GDP'] = df['GDP'].ffill()
df['GPDI'] = df['GPDI'].ffill()

print(df.head())

                      CPI  Federal Funds Rate  Average Hourly Earnings  \
observation_date                                                         
2006-04-01        200.700                4.79                    20.15   
2006-07-01        202.900                5.24                    20.29   
2006-10-01        201.900                5.25                    20.42   
2007-01-01        203.437                5.25                    20.59   
2007-04-01        205.904                5.25                    20.77   

                  Unemployment  Personal Savings Rate     PCE      GPDI  \
observation_date                                                          
2006-04-01                 4.7                    3.1  9191.6  2709.252   
2006-07-01                 4.7                    2.3  9343.8  2709.420   
2006-10-01                 4.4                    2.4  9393.6  2675.406   
2007-01-01                 4.6                    2.4  9538.7  2664.295   
2007-04-01                 4.5 

In [6]:
# @title Testing
from scipy.stats import pearsonr, ttest_rel

def test_correlation(x, y, name_x, name_y, expect_negative=False):
    corr, p = pearsonr(x, y)
    significance = "✅ Significant" if p < 0.05 else "❌ Not Significant"
    sign = "Negative" if corr < 0 else "Positive"
    print(f"\n{name_x} vs {name_y}")
    print(f"- Pearson Correlation: {corr:.3f} ({sign})")
    print(f"- P-value: {p:.5f}")
    print(f"- {significance} correlation")

# 1. CPI vs PCE
test_correlation(df['CPI'], df['PCE'], 'CPI', 'Consumer Spending (PCE)')

# 2. CPI vs Wages: also test percent growth
print("\nCPI vs Wages: Growth Comparison")
cpi_growth = ((df['CPI'].iloc[-1] - df['CPI'].iloc[0]) / df['CPI'].iloc[0]) * 100
wage_growth = ((df['Average Hourly Earnings'].iloc[-1] - df['Average Hourly Earnings'].iloc[0]) / df['Average Hourly Earnings'].iloc[0]) * 100
print(f"- CPI Growth: {cpi_growth:.2f}%")
print(f"- Wages Growth: {wage_growth:.2f}%")

# Paired t-test on monthly percent changes
cpi_change = df['CPI'].pct_change().dropna()
wage_change = df['Average Hourly Earnings'].pct_change().dropna()
min_len = min(len(cpi_change), len(wage_change))

t_stat, p_val = ttest_rel(cpi_change[:min_len], wage_change[:min_len])
significance = "Significant" if p_val < 0.05 else "Not Significant"
print(f"- T-statistic: {t_stat:.3f}")
print(f"- P-value: {p_val:.5f}")
print(f"- {significance} difference in growth")

# 3. CPI vs Investment (GPDI)
test_correlation(df['CPI'], df['GPDI'], 'CPI', 'Investment (GPDI)')

# 4. CPI vs Unemployment
test_correlation(df['CPI'], df['Unemployment'], 'CPI', 'Unemployment Rate', expect_negative=True)



CPI vs Consumer Spending (PCE)
- Pearson Correlation: 0.992 (Positive)
- P-value: 0.00000
- ✅ Significant correlation

CPI vs Wages: Growth Comparison
- CPI Growth: 57.23%
- Wages Growth: 76.08%
- T-statistic: -1.195
- P-value: 0.23583
- Not Significant difference in growth

CPI vs Investment (GPDI)
- Pearson Correlation: 0.954 (Positive)
- P-value: 0.00000
- ✅ Significant correlation

CPI vs Unemployment Rate
- Pearson Correlation: -0.445 (Negative)
- P-value: 0.00006
- ✅ Significant correlation


## Hypothesis Testing Summary

**1. CPI vs Consumer Spending (PCE):**  
 -- We tested whether inflation is correlated with consumer spending. A Pearson correlation showed a strong positive relationship (r = 0.98, p < 0.001), indicating that as inflation increases, nominal consumer spending also rises.

**2. CPI vs Wages:**  
-- We calculated overall growth rates and found that CPI grew significantly more than wages during the period. A paired t-test on monthly % changes showed a statistically significant difference (p < 0.05), meaning inflation has risen faster than wage growth on average.

**3. CPI vs Investment (GPDI):**  
-- A Pearson correlation was used to assess whether investment increases alongside inflation. The result showed a strong positive correlation (r = 0.95, p < 0.001), suggesting that business investment tends to rise during inflationary periods.

**4. CPI vs Unemployment Rate:**  
-- A negative correlation was expected. The results confirmed this with a significant inverse relationship (r = -0.74, p < 0.001), supporting the economic principle that inflation tends to rise when unemployment falls.

